# RAG(Retrieval-Augmented-Generation) Client

## 1. Download Model

```bash
gai pull instructor-sentencepiece
```

## 2. Start RAG Service

* Go to /gai

* Open docker-compose.yml

* Make sure that the following section is not commented.

```yaml
    gai-rag-svr:
        image: kakkoii1337/gai-rag-svr:1.0.17
        container_name: gai-rag-svr
        ...
```

* Run the command:

```sh
docker-compose up
```



## Example 1: Ask about a Web Page

### Step 1: Scrape text from the web for testing

Scrape a speech from the web and save it to a file.

In [2]:
import os
os.environ["LOG_LEVEL"]="INFO"
from gai.lib.tools.scraper import Scraper
scraper=Scraper()
text,links = scraper.scrape("https://www.pmo.gov.sg/Newsroom/national-day-rally-2023")
import tempfile
with tempfile.NamedTemporaryFile(mode="w", delete=False) as f:
    f.write(text)
    file_path = f.name

### Step 2: Index the file

In [3]:
# Load RAG client
from gai.rag.client.rag_client_async import RagClientAsync
rag = RagClientAsync({
        "type": "rag",
        "url": "http://localhost:12036/gen/v1/rag",
        "ws_url": "ws://localhost:12036/gen/v1/rag/index-file/ws"
})

# Create callback
import json
async def async_callback(text):
    jsoned = json.loads(text)
    print(f"async_callback: {jsoned}")

# Index the sample speech
result = await rag.index_document_async(
    collection_name="demo",
    file_path=file_path,
    file_type="txt",
    title="2023 National Day Rally Speech",
    source="https://www.pmo.gov.sg/Newsroom/national-day-rally-2023",
    async_callback=async_callback,
)


### Step 3. Asking a question

In [4]:
# Load RAG client
from gai.rag.client.rag_client_async import RagClientAsync
rag = RagClientAsync({
        "type": "rag",
        "url": "http://localhost:12036/gen/v1/rag",
        "ws_url": "ws://localhost:12036/gen/v1/rag/index-file/ws"
})

# Retrieve from query
results = await rag.retrieve_async(
    collection_name="demo",
    query_texts="Who are the young seniors?",
    n_results=1
)
for result in results:
    print(result)


{'documents': 'and the Merdeka Generation; “Seniors”, because you will soon retire, or maybe you have recently retired. Young Seniors are in a unique position today. Compared to the Pioneer and Merdeka Generations, you have benefited more from Singapore’s growth, and generally done better in life. But compared to workers younger than you, in their 30s and 40s today, you have generally earned less over your lifetimes. You have also had less time to benefit from improvements to the CPF system, and so have built up less retirement savings. Young Seniors are also in a particularly sandwiched phase of your lives. You have to shoulder the responsibility of caring for both the young and old in your families. Your kids may be young adults, but often are not yet fully independent and still live in the same household. Many of you Young Seniors also have elderly parents at home, who may be beset with the infirmities of old age. You have to shuttle them to medical appointments and hospital visits,

### 3. List Collections

In [5]:
# Load RAG client
from gai.rag.client.rag_client_async import RagClientAsync
rag = RagClientAsync({
        "type": "rag",
        "url": "http://localhost:12036/gen/v1/rag",
        "ws_url": "ws://localhost:12036/gen/v1/rag/index-file/ws"
})

await rag.list_collections_async()

{'collections': ['demo']}

### 4. List Document Headers

In [6]:
docs = await rag.list_documents_async(collection_name="demo")
docs

[IndexedDocPydantic(Id='F5q0Xq9WTANXAuKfCj-DRvEDkRmx1e7kJniKwOuV8Ec', CollectionName='demo', ByteSize=44190, FileName='tmphfvm6wkb', FileType='txt', File=None, Source='https://www.pmo.gov.sg/Newsroom/national-day-rally-2023', Abstract=None, Authors='', Title='2023 National Day Rally Speech', Publisher='', PublishedDate=None, Comments='', Keywords='', CreatedAt=datetime.datetime(2024, 12, 18, 14, 48, 42, 800804), UpdatedAt=datetime.datetime(2024, 12, 18, 14, 48, 42, 800810), IsActive=True, ChunkGroups=[IndexedDocChunkGroupPydantic(Id='72ec0531-d8d8-4eb0-aa59-c7e859ab3fe8', DocumentId='F5q0Xq9WTANXAuKfCj-DRvEDkRmx1e7kJniKwOuV8Ec', SplitAlgo='recursive_split', ChunkCount=49, ChunkSize=1000, Overlap=100, IsActive=True, ChunksDir='/tmp/chunks/2178654ed97a4b3f9041471431c62088')])]

### 5. Download File

In [7]:
await rag.get_document_file_async(collection_name="demo",document_id=docs[0].Id)

---

## Appendix: Multi-Step Indexing

This achieves the same outcome as step 2 but broken down into multiple steps for ease of debugging. For this demo, we shall use a PDF file.

### Step 0: Setup

In [8]:
import os
from gai.rag.client.rag_client_async import RagClientAsync

rag = RagClientAsync({
    "type": "rag",
    "url": "http://localhost:12036/gen/v1/rag",
    "ws_url": "ws://localhost:12036/gen/v1/rag/index-file/ws",
})


### Step 1: Create header

In [9]:
here = os.path.dirname(__name__)
file_path=os.path.join(here,"ReAct-2210.03629.pdf")
doc_header = await rag.step_header_async(
    collection_name="demo",
    file_path=file_path,
    title="ReAct: Synergizing Reasoning and Acting in Language Models",
    source="https://arxiv.org/abs/2210.03629",
)
doc_header

IndexedDocPydantic(Id='Mm1GAmuGfs3VoA2sit0njabQGAZ75eyfszIkAxGY5e4', CollectionName='demo', ByteSize=633805, FileName='ReAct-2210.03629.pdf', FileType='.pdf', File=None, Source='https://arxiv.org/abs/2210.03629', Abstract=None, Authors='', Title='ReAct: Synergizing Reasoning and Acting in Language Models', Publisher='', PublishedDate=None, Comments='', Keywords='', CreatedAt=datetime.datetime(2024, 12, 18, 14, 49, 34, 954703), UpdatedAt=datetime.datetime(2024, 12, 18, 14, 49, 34, 954711), IsActive=True, ChunkGroups=[])

### Step 2: Split Document

In [10]:
chunkgroup = await rag.step_split_async(
    collection_name="demo",
    document_id=doc_header.Id,
    chunk_size=1000,
    chunk_overlap=100
)
chunkgroup

IndexedDocChunkGroupPydantic(Id='e73a0b4b-aa71-4e55-96c1-d9cf9f1a148a', DocumentId='Mm1GAmuGfs3VoA2sit0njabQGAZ75eyfszIkAxGY5e4', SplitAlgo='recursive_split', ChunkCount=114, ChunkSize=1000, Overlap=100, IsActive=True, ChunksDir='/tmp/chunks/9c6016d17e4245fcae89057c7ff41bd2')

### Step 3: Index Chunks

In [11]:
# Create callback
import json
async def async_callback(text):
    jsoned = json.loads(text)
    print(f"async_callback: {jsoned}")

# Spin off listener
from gai.lib.common.StatusListener import StatusListener
ws_url = f"ws://localhost:12036/gen/v1/rag/index-file/ws/demo"
listener = StatusListener(ws_url)
import asyncio
listen_task=asyncio.create_task(listener.listen(async_callback))

result = await rag.step_index_async(
    collection_name="demo",
    document_id=chunkgroup.DocumentId,
    chunkgroup_id=chunkgroup.Id,
    async_callback=async_callback
)
result


async_callback: {'progress': 0}
async_callback: {'progress': 1}
async_callback: {'progress': 2}
async_callback: {'progress': 3}
async_callback: {'progress': 4}
async_callback: {'progress': 5}
async_callback: {'progress': 6}
async_callback: {'progress': 7}
async_callback: {'progress': 7}
async_callback: {'progress': 8}
async_callback: {'progress': 9}
async_callback: {'progress': 10}
async_callback: {'progress': 11}
async_callback: {'progress': 12}
async_callback: {'progress': 13}
async_callback: {'progress': 14}
async_callback: {'progress': 14}
async_callback: {'progress': 15}
async_callback: {'progress': 16}
async_callback: {'progress': 17}
async_callback: {'progress': 18}
async_callback: {'progress': 19}
async_callback: {'progress': 20}
async_callback: {'progress': 21}
async_callback: {'progress': 21}
async_callback: {'progress': 22}
async_callback: {'progress': 23}
async_callback: {'progress': 24}
async_callback: {'progress': 25}
async_callback: {'progress': 26}
async_callback: {'pro

IndexedDocChunkIdsPydantic(DocumentId='Mm1GAmuGfs3VoA2sit0njabQGAZ75eyfszIkAxGY5e4', ChunkgroupId='e73a0b4b-aa71-4e55-96c1-d9cf9f1a148a', ChunkIds=['7c1033ae-e78b-4577-8683-561973c39fa5', '331dfac4-a4aa-42a4-80a3-18168e905457', '33e214e1-c02e-4853-b08f-555fcd32c322', '334eff67-8839-4c56-bb91-4601eaa2c481', '54603927-d461-4507-a82f-f5497097873f', '04c2bf17-e735-438d-89c9-5c3596e755d4', '685b63f1-a7db-438c-b1ad-2292c2981e6f', '11557f61-06a1-4079-95a6-c823820f0d27', 'd2d4184f-16e4-4a95-9379-6f3b69efbacd', 'd3d6eb42-c7fc-4492-a1a6-4129ff518937', '2c65949c-c7e4-438a-8c52-197692d1784e', '6620b473-d4dd-4cfa-889a-a628199949c5', '59bddd8f-2407-4b0f-8846-f6d79eba373d', '6318e9ad-17c2-4714-b867-2bfd401c5774', '479a397c-76e8-4181-b23b-d5c4e3933e95', 'c80c6e7e-29f7-4a12-8e67-61d378026fe3', '8a7353e7-c736-4dcd-aa7e-f458220eecad', '00054956-d99b-44a1-a866-ae12f696fb04', '528e0c6f-3424-4fe6-beee-5bccee7d30bd', 'a8156a93-92ae-46f4-a39a-ad801c14b772', 'f2574ac3-0959-459d-8e3c-86340b920946', '99dff46f-15

WARNING  StatusListener.listen: Server disconnected.
